<a href="https://colab.research.google.com/github/Charith-25/RL-Project/blob/main/RL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.6 MB/s eta 0:00:00


In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import SingleGrid
import random

class GarbageTruck(Agent):
    def __init__(self, unique_id, model, speed, capacity, routing_preferences):
        super().__init__(unique_id, model)
        self.speed = speed
        self.capacity = capacity
        self.routing_preferences = routing_preferences
        self.current_load = 0

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def collect_garbage(self):
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for agent in cell_contents:
            if isinstance(agent, Garbage):
                if self.current_load < self.capacity:
                    self.model.grid.remove_agent(agent)
                    self.current_load += 1

    def step(self):
        self.move()
        self.collect_garbage()

class Garbage(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

class GarbageModel(Model):
    def __init__(self, num_trucks, width, height):
        self.num_trucks = num_trucks
        self.grid = SingleGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)

        # Create garbage trucks
        for i in range(num_trucks):
            speed = random.randint(1, 5)
            capacity = random.randint(10, 20)
            routing_preferences = random.choice(['A', 'B', 'C'])
            truck = GarbageTruck(i, self, speed, capacity, routing_preferences)
            self.schedule.add(truck)
            # Place trucks randomly on the grid
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(truck, (x, y))

        # Create garbage agents
        for i in range(width * height // 10):
            garbage = Garbage(i + num_trucks, self)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(garbage, (x, y))
            self.schedule.add(garbage)

    def step(self):
        self.schedule.step()

# Create the model with 5 trucks in a 10x10 grid
model = GarbageModel(num_trucks=5, width=10, height=10)

# Run the simulation for 10 steps
for i in range(10):
    model.step()


ModuleNotFoundError: ignored